Dans la fonction `calc_log_pobs_v2`, il nous faut calculer $\alpha_t(j)$ en utilisant la formule récursive
$$\alpha_t(j) = \left(\sum_{i=0}^{N-1} \alpha_{t-1}(i) a_{ij}\right) b_{j x_t},$$
mais le tableau `alpha` qu'on a stocke uniquement $\log\alpha_{t-1}(i)$, car les $\alpha$ peuvent être trop petits et donc non représentables en tant que flottants en Python. En plus, appliquer le log des deux côtés de cette expression donne
$$\log \alpha_t(j) = \log\left(\sum_{i=0}^{N-1} \alpha_{t-1}(i) a_{ij}\right) + \log b_{j x_t}$$
mais il n'y a pas de formule simple pour le logarithme d'une somme. Pour calculer le logarithme de cette somme, on utilise alors la technique suivante :

Supposons qu'on a une somme $\sum_{i=0}^{N-1} x_i$ à calculer mais on dispose uniquement d'un tableau `aux` contenant $\log x_i$. On peut alors calculer
$$
\log\left(\sum_{i=0}^{N-1} x_i\right) = \log\left(x_{\max}\sum_{i=0}^{N-1} \frac{x_i}{x_{\max}}\right) = \log x_{\max} + \log \left(\sum_{i=0}^{N-1} \frac{x_i}{x_{\max}}\right).
$$
Pour ce dernier $\log$, on peut faire
$$
\log \left(\sum_{i=0}^{N-1} \frac{x_i}{x_{\max}}\right) = \log \left(\sum_{i=0}^{N-1} \frac{e^{\log x_i}}{e^{\log x_{\max}}}\right) = \log \left(\sum_{i=0}^{N-1} e^{\log x_i - \log x_{\max}}\right)
$$
Alors
$$
\log\left(\sum_{i=0}^{N-1} x_i\right) = \log x_{\max} + \log \left(\sum_{i=0}^{N-1} e^{\log x_i - \log x_{\max}}\right)
$$
et on peut donc calculer cela avec le tableau `aux` comme
$$
\log\left(\sum_{i=0}^{N-1} x_i\right) = \text{aux.max()} + \log \left(\sum_{i=0}^{N-1} e^{\text{aux[i]} - \text{aux.max()}}\right).
$$
L'avantage de cette méthode est qu'on ne calcule jamais les $x_i$ eux-mêmes, qui peuvent être trop petits pour être représentés en tant que flottants, mais on calcule uniquement $\frac{x_i}{x_{\max}} = e^{\text{aux[i]} - \text{aux.max()}}$, qui, en tant que rapport de deux nombres petits, n'est pas forcément petit.

Il y a un seul détail à régler : si `aux.max()==-np.inf`, alors ce calcul donne `nan`. Mais, dans ce cas, cela veut dire que $x_i = 0$ pour tout $i$ et donc le log de la somme qu'on veut calculer est `-np.inf`.